In [1]:
import pandas as pd
import json

df_csv = pd.read_csv('UNSPSC_English.csv')
df_xlsx = pd.read_excel('Avalara_goods_and_services.xlsx')

/tmp/ipykernel_67896/4190874567.py:4: DtypeWarning: Columns (1,2,5,8,9,12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv('UNSPSC_English.csv')


In [2]:
# Convert row to header and remove the row
df_csv = df_csv.rename(columns=df_csv.iloc[8]).loc[9:100]
df_csv = df_csv.drop(columns = ['Version', 'Key', 'Column1', 'Synonym', 'Acronym'])
df_csv

,Segment,Segment Title,Segment Definition,Family,Family Title,Family Definition,Class,Class Title,Class Definition,Commodity,Commodity Title,Definition
9,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,NaN,NaN,NaN
12,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101501,Cats,NaN
13,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101502,Dogs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
96,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10102000,Wild animals,NaN,10102002,Live foxes,NaN
97,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10102000,Wild animals,NaN,10102003,Live bothrops pit viper snake,A venomous pit viper snake spanning 32 species...
98,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10102000,Wild animals,NaN,10102004,Live chironius or vine snake,A type of tree-dwelling snake spanning 13 spec...
99,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10102000,Wild animals,NaN,10102005,Live clelia or mussurana snake,A type of snake spanning 10 species whose prey...


In [3]:
df_xlsx = df_xlsx.rename(columns={'Unnamed: 0':'AvaTax System Tax Code', 'AvaTax System Tax Codes':'AvaTax System Tax Code Description',
                                 'Unnamed: 2':'Additional AvaTax System Tax Code Information'})
df_xlsx = df_xlsx[2:]
df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information
2,D0000000,Digital goods,Digital goods are generally viewed as download...
3,D9999999,Temporary unmapped digital goods sku - taxable...,A digital good that has not been mapped to a s...
4,DA010000,Digital goods: advertising material (digital o...,"Tangible personal property (i.e. designs, artw..."
5,DA030000,Digital goods: art (digital only -- no physica...,Canned artwork that is transferred to the cust...
6,DA040000,Digital goods: audio (digital only -- no physi...,Audio (i.e. music or spoken word) that is tran...
...,...,...,...
2541,SY010200,Yoga classes - in facilities that offer servic...,NaN
2542,SL090200,Lodging accommodations,NaN
2543,ST110000,Travel agency,NaN
2544,ST112506,Travel agency fees,NaN


In [4]:
# import nltk and download resources
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/diz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/diz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# parse the stop words and special characters out
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
stop_words.add(':')
stop_words.add('--')
stop_words.add('-')
stop_words.add('(')
stop_words.add(')')
stop_words.add('/')
stop_words.add('*')
stop_words.add(';')
stop_words.add('.')
stop_words.add('i.e')
stop_words.add('A')
stop_words.add('a')

# clean col 1
column_name = 'AvaTax System Tax Code Description'
for index, description in enumerate(df_xlsx[column_name], 2):
    if not pd.isna(description):
        word_tokens = word_tokenize(description)

        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w.lower())

        # remove empty strings
        while "" in filtered_sentence:
            filtered_sentence.remove("")

        # updating record in AvaTax System Tax Code Description
        df_xlsx.at[index, column_name]= ','.join(filtered_sentence)

# clean col 2
column_name_2 = 'Additional AvaTax System Tax Code Information'
for index, description in enumerate(df_xlsx[column_name_2], 2):
    if not pd.isna(description):
        word_tokens = word_tokenize(description)

        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w.lower())

        # remove empty strings
        while "" in filtered_sentence:
            filtered_sentence.remove("")

        # updating record in AvaTax System Tax Code Description
        df_xlsx.at[index, column_name_2]= ','.join(filtered_sentence)

# create 3rd column
new_column_series = []
for index, description in enumerate(df_xlsx[column_name_2], 2):
    temp1 = df_xlsx[column_name].loc[index].split(',')

    if not pd.isna(description):
        temp2 = df_xlsx[column_name_2].loc[index].split(',')
        temp3 = temp1 + temp2
        temp3 = set(temp3)

        if '' in temp3:
            temp3.remove('')

        new_column_series.append(list(temp3))
    else:
        new_column_series.append(list(temp1))

df_xlsx.loc[:,'new_column'] = new_column_series

df_xlsx.drop(column_name, axis=1, inplace=True)
df_xlsx.drop(column_name_2, axis=1, inplace=True)

df_xlsx

,AvaTax System Tax Code,new_column
2,D0000000,"[artwork, would, music, files, otherwise, phot..."
3,D9999999,"[exemption, accurately, using, reviewed, adequ..."
4,DA010000,"[property, artwork, transferred, media, servic..."
5,DA030000,"[artwork, canned, transferred, customer, art, ..."
6,DA040000,"[transferred, customer, music, digital, goods,..."
...,...,...
2541,SY010200,"[yoga, classes, facilities, offer, services, y..."
2542,SL090200,"[lodging, accommodations]"
2543,ST110000,"[travel, agency]"
2544,ST112506,"[travel, agency, fees]"


In [6]:
tc_hash_map = {}

for index, tax_code in enumerate(df_xlsx['AvaTax System Tax Code'], 2):
    
    # check if tax code exists
    
    #if tax_code == "":
    if isinstance(tax_code, float):
        prefix = df_xlsx["AvaTax System Tax Code"].loc[index + 1]
        
        level_1 = prefix[0:2]
        level_2 = prefix[2:4]
        level_3 = "0000"
    
        print(tax_code)
        
        df_xlsx["AvaTax System Tax Code"].loc[index] = prefix[0:4]+"0000"
        
    else: 
        
        level_1 = tax_code[0:2]
        level_2 = tax_code[2:4]
        level_3 = tax_code[4:8]
    

    if level_1 not in tc_hash_map:
        temp_dict0 = {level_3: df_xlsx['new_column'].loc[index]}
        temp_dict = {level_2: [df_xlsx['new_column'].loc[index], temp_dict0]}
        tc_hash_map[level_1] = [df_xlsx['new_column'].loc[index], temp_dict]
    elif level_2 not in tc_hash_map[level_1][1]:
        temp_dict0 = {level_3: df_xlsx['new_column'].loc[index]}
        tc_hash_map[level_1][1][level_2] = [df_xlsx['new_column'].loc[index], temp_dict0]
    elif level_3 not in tc_hash_map[level_1][1][level_2][1]:
        tc_hash_map[level_1][1][level_2][1][level_3] = df_xlsx['new_column'].loc[index]

print(json.dumps(tc_hash_map, sort_keys=True, indent=2))

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
{
  "D0": [
    [
      "artwork",
      "would",
      "music",
      "files",
      "otherwise",
      "photographs",
      "viewed",
      "transferred",
      "examples",
      "movies",
      "websites",
      "electronically",
      "include",
      "generally",
      "goods",
      "zip",
      "items",
      "e-books",
      "downloadable",
      "digital",
      "sold",
      "pdfs",
      "software",
      "computer"
    ],
    {
      "00": [
        [
          "artwork",
          "would",
          "music",
          "files",
          "otherwise",
          "photographs",
          "viewed",
          "transferred",
          "examples",
          "movies",
          "websites",
          "electronically",
          "include",
          "generally",
          "goods",
          "zip",
          "items",
          "e-books",
          "downloadable",
          "digital",
      